Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---
Answer(for logistic): the line is added is: loss += 3e-3*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
best result : 82.1 and 88.6 for validation and test accuracy respectively.
for neural networks : the lines added: 
  regularizers = (tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(biases['b1']) +
                  tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(biases['b2']))
  loss += 3e-3 * regularizers
best results: 86.3 and 92.2 

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
  # line I added for regularization 
  loss += 3e-3*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [131]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 25.889942
Minibatch accuracy: 12.5%
Validation accuracy: 14.3%
Minibatch loss at step 500: 2.830688
Minibatch accuracy: 79.7%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 1.049459
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.640823
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.695376
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.841229
Minibatch accuracy: 75.8%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.469724
Minibatch accuracy: 88.3%
Validation accuracy: 82.1%
Test accuracy: 88.6%


In [132]:
batch_size = 128
hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights={  
      'w1' : tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden])),
      'w2' : tf.Variable(
        tf.truncated_normal([hidden, num_labels]))
  }      
        
  biases={      
      'b1' : tf.Variable(tf.zeros([hidden])),

      'b2' : tf.Variable(tf.zeros([num_labels]))
  }  
  
  # Training computation.
  def compute_pred(input, weights, biases):
      hidden_layer = tf.matmul(input, weights['w1']) + biases['b1']
      hidden_layer = tf.nn.relu(hidden_layer)
      last_layer = tf.matmul(hidden_layer, weights['w2']) + biases['b2']
      return last_layer 
    
  logits = compute_pred(tf_train_dataset, weights, biases)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  regularizers = (tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(biases['b1']) +
                  tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(biases['b2']))

  # Add the regularization term to the loss.
  loss += 3e-3 * regularizers

  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute_pred(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(compute_pred(tf_test_dataset, weights, biases))

In [45]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 452.622223
Minibatch accuracy: 12.5%
Validation accuracy: 21.2%
Minibatch loss at step 500: 130.929367
Minibatch accuracy: 77.3%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 96.677315
Minibatch accuracy: 83.6%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 72.670715
Minibatch accuracy: 86.7%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 58.017456
Minibatch accuracy: 86.7%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 44.684380
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 34.295219
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Test accuracy: 91.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [126]:
batch_size = 250
hidden = 1024 

graph = tf.Graph()
with graph.as_default():
  dropout = 0.60
  keep_prob = tf.placeholder(tf.float32)
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights={  
      'w1' : tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden], stddev=0.04)),
      'w2' : tf.Variable(
        tf.truncated_normal([hidden, num_labels], stddev=0.065))
  }      
        
  biases={      
      'b1' : tf.Variable(tf.zeros([hidden])),

      'b2' : tf.Variable(tf.zeros([num_labels]))
  }  
  
  # Training computation.
  def compute_pred(input, weights, biases, dropout):
      hidden_layer = tf.matmul(input, weights['w1']) + biases['b1']
      hidden_layer = tf.nn.relu(hidden_layer)
      #I implemented dropout here
      hidden_layer = tf.nn.dropout(hidden_layer, dropout)
      last_layer = tf.matmul(hidden_layer, weights['w2']) + biases['b2']
      return last_layer 
    
  logits = compute_pred(tf_train_dataset, weights, biases, keep_prob)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  #regularizers = (tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(biases['b1']) +
  #                tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(biases['b2']))

   #Add the regularization term to the loss.
  #loss += 3e-3 * regularizers

  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute_pred(tf_valid_dataset, weights, biases, 1.0))
  test_prediction = tf.nn.softmax(compute_pred(tf_test_dataset, weights, biases, 1.0))

In [127]:
num_steps = 3001
import time
t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : dropout}
    session.run(
      [optimizer], feed_dict=feed_dict)
    if (step % 500 == 0):
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0}
      l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
t1 = time.time()
print ("time: ", t1-t0)

Initialized
Minibatch loss at step 0: 1.341327
Minibatch accuracy: 59.2%
Validation accuracy: 46.0%
Minibatch loss at step 500: 0.312837
Minibatch accuracy: 91.6%
Validation accuracy: 86.5%
Minibatch loss at step 1000: 0.229538
Minibatch accuracy: 94.0%
Validation accuracy: 87.1%
Minibatch loss at step 1500: 0.332626
Minibatch accuracy: 92.0%
Validation accuracy: 88.3%
Minibatch loss at step 2000: 0.261099
Minibatch accuracy: 92.4%
Validation accuracy: 88.6%
Minibatch loss at step 2500: 0.145998
Minibatch accuracy: 96.4%
Validation accuracy: 89.0%
Minibatch loss at step 3000: 0.241239
Minibatch accuracy: 94.8%
Validation accuracy: 88.9%
Test accuracy: 94.4%
time:  166.850914955


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## This is using rate decay

In [114]:
batch_size = 250
hidden = 1024

graph = tf.Graph()
with graph.as_default():
  dropout = 0.50
  keep_prob = tf.placeholder(tf.float32)
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights={  
      'w1' : tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden], stddev=0.04)),
      'w2' : tf.Variable(
        tf.truncated_normal([hidden, num_labels], stddev=0.065))
  }      
        
  biases={      
      'b1' : tf.Variable(tf.zeros([hidden])),

      'b2' : tf.Variable(tf.zeros([num_labels]))
  }  
  
  # Training computation.
  def compute_pred(input, weights, biases, dropout):
      hidden_layer = tf.matmul(input, weights['w1']) + biases['b1']
      hidden_layer = tf.nn.relu(hidden_layer)
      #I implemented dropout here
      hidden_layer = tf.nn.dropout(hidden_layer, dropout)
      last_layer = tf.matmul(hidden_layer, weights['w2']) + biases['b2']
      return last_layer 
    
  logits = compute_pred(tf_train_dataset, weights, biases, keep_prob)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  regularizers = (tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(biases['b1']) +
                  tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(biases['b2']))

   #Add the regularization term to the loss.
  loss += 3e-3 * regularizers

  

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute_pred(tf_valid_dataset, weights, biases, 1.0))
  test_prediction = tf.nn.softmax(compute_pred(tf_test_dataset, weights, biases, 1.0))

In [115]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : dropout}
    session.run(
      [optimizer], feed_dict=feed_dict)
    if (step % 500 == 0):
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0}
      l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.720979
Minibatch accuracy: 66.0%
Validation accuracy: 53.3%
Minibatch loss at step 500: 0.803907
Minibatch accuracy: 88.4%
Validation accuracy: 85.4%
Minibatch loss at step 1000: 0.518067
Minibatch accuracy: 91.2%
Validation accuracy: 85.8%
Minibatch loss at step 1500: 0.576829
Minibatch accuracy: 90.0%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 0.516984
Minibatch accuracy: 86.4%
Validation accuracy: 86.2%
Minibatch loss at step 2500: 0.402618
Minibatch accuracy: 92.8%
Validation accuracy: 86.4%
Minibatch loss at step 3000: 0.536148
Minibatch accuracy: 87.6%
Validation accuracy: 86.5%
Test accuracy: 92.3%


## The next two chuncks of code are using multi layer neural network model

In [8]:
batch_size = 128
hidden1 = 1024
hidden2 = 784

graph = tf.Graph()
with graph.as_default():
  dropout = 0.60
  keep_prob = tf.placeholder(tf.float32)
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights={  
      'w1' : tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden1], stddev=0.04)),
      'w2' : tf.Variable(
        tf.truncated_normal([hidden1, hidden2], stddev=0.04)),    
      'w3' : tf.Variable(
        tf.truncated_normal([hidden2, num_labels], stddev=0.08))
  }      
        
  biases={      
      'b1' : tf.Variable(tf.zeros([hidden1])),
      'b2' : tf.Variable(tf.zeros([hidden2])),
      'b3' : tf.Variable(tf.zeros([num_labels]))
  }  
  
  # Training computation.
  def compute_pred(input, weights, biases, dropout):
      hidden_layer1 = tf.matmul(input, weights['w1']) + biases['b1']
      hidden_layer1 = tf.nn.relu(hidden_layer1)
      #I implemented dropout here
      hidden_layer1 = tf.nn.dropout(hidden_layer1, dropout)
      hidden_layer2 = tf.matmul(hidden_layer1, weights['w2']) + biases['b2']
      hidden_layer2 = tf.nn.relu(hidden_layer2)      
      hidden_layer2 = tf.nn.dropout(hidden_layer2, dropout)        
      last_layer = tf.matmul(hidden_layer2, weights['w3']) + biases['b3']
      return last_layer 
    
  logits = compute_pred(tf_train_dataset, weights, biases, keep_prob)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  #regularizers = (tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(biases['b1']) +
  #                tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(biases['b2']) +
  #                tf.nn.l2_loss(weights['w3']) + tf.nn.l2_loss(biases['b3']))

   #Add the regularization term to the loss.
  #loss += 3e-3 * regularizers

  

  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute_pred(tf_valid_dataset, weights, biases, 1.0))
  test_prediction = tf.nn.softmax(compute_pred(tf_test_dataset, weights, biases, 1.0))

In [9]:
num_steps = 95000
import time
t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : dropout}
    session.run(
      [optimizer], feed_dict=feed_dict)
    if (step % 500 == 0):
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0}
      l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
t1 = time.time()
print ("time: ", t0-t1)

Initialized
Minibatch loss at step 0: 1.336504
Minibatch accuracy: 64.8%
Validation accuracy: 46.9%
Minibatch loss at step 500: 0.360844
Minibatch accuracy: 89.1%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.373197
Minibatch accuracy: 91.4%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 0.235315
Minibatch accuracy: 92.2%
Validation accuracy: 87.0%
Minibatch loss at step 2000: 0.292679
Minibatch accuracy: 93.0%
Validation accuracy: 87.5%
Minibatch loss at step 2500: 0.347360
Minibatch accuracy: 89.8%
Validation accuracy: 88.3%
Minibatch loss at step 3000: 0.202194
Minibatch accuracy: 96.1%
Validation accuracy: 88.3%
Minibatch loss at step 3500: 0.240816
Minibatch accuracy: 93.0%
Validation accuracy: 88.6%
Minibatch loss at step 4000: 0.227448
Minibatch accuracy: 94.5%
Validation accuracy: 88.7%
Minibatch loss at step 4500: 0.194191
Minibatch accuracy: 95.3%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.243759
Minibatch accuracy: 93.8%
Validation accurac